In [ ]:
key_vault_name = 'kv_to-be-replaced'

In [ ]:
from trident_token_library_wrapper import PyTridentTokenLibrary as tl

def get_secrets_from_kv(kv_name, secret_name):

    access_token = mssparkutils.credentials.getToken("keyvault")
    kv_endpoint = f'https://{kv_name}.vault.azure.net/'
    return(tl.get_secret_with_token(kv_endpoint,secret_name,access_token))

openai_api_type = "azure"
openai_api_version  = get_secrets_from_kv(key_vault_name,"AZURE-OPENAI-VERSION")
openai_api_base = get_secrets_from_kv(key_vault_name,"AZURE-OPENAI-ENDPOINT")
openai_api_key = get_secrets_from_kv(key_vault_name,"AZURE-OPENAI-KEY")

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, MapType, LongType, TimestampType
import os

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, MapType, LongType, TimestampType
import os

folder_path = 'Files/data/conversation_input/'

# Define the schema for the nested Messages in the Conversation
message_schema = StructType([
    StructField("Id", StringType(), True),
    StructField("ReferenceId", StringType(), True),
    StructField("EventType", StringType(), True),
    StructField("EventTime", StringType(), True),
    StructField("ConversationId", StringType(), True),
    StructField("Value", StringType(), True),
    StructField("UserId", StringType(), True),
    StructField("CustomProperties", MapType(StringType(), StringType()), True)
])

# Define the schema for the Conversation
conversation_schema = StructType([
    StructField("ConversationId", StringType(), True),
    StructField("Messages", ArrayType(message_schema), True),
    StructField("StartTime", TimestampType(), True),
    StructField("EndTime", TimestampType(), True),
    StructField("Merged_content", StringType(), True),
    StructField("Merged_content_user", StringType(), True),
    StructField("Merged_content_agent", StringType(), True),
    StructField("Full_conversation", StringType(), True),
    StructField("Duration", LongType(), True)  # New field for duration
])

# Define the complete schema for the JSON document
schema = StructType([
    StructField("AgentName", StringType(), True),
    StructField("AgentId", StringType(), True),
    StructField("Team", StringType(), True),
    StructField("ResolutionStatus", StringType(), True),
    StructField("CallReason", StringType(), True),
    StructField("CallerID", StringType(), True),
    StructField("Conversation", conversation_schema, True)
])

df = None
df = spark.read.option("multiLine", True).schema(schema).option("mode", "FAILFAST").json(folder_path)

# Print the schema to confirm it matches the intended design
# df.printSchema()

#use the legacy time parser policy
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

# Update Duration field with the duration from StartTime to Endtime in milliseconds
df = df.withColumn("Conversation", df["Conversation"].withField("Duration", 
                                                (F.unix_timestamp(df["Conversation"]["EndTime"], 'yyyy-MM-dd\'T\'HH:mm:ss') - 
                                                 F.unix_timestamp(df["Conversation"]["StartTime"], 'yyyy-MM-dd\'T\'HH:mm:ss')) / 60))

df = df.withColumn("Conversation", df["Conversation"].withField("ConversationDate", (df["Conversation"]["StartTime"])))

df.select("Conversation.*").show()

df.printSchema()

In [ ]:
# Select specific columns, including nested ones
selected_df = df.select(
    "AgentName",
    "AgentId",
    "Team",
    "ResolutionStatus",
    "CallReason",
    "CallerID",
    "Conversation.ConversationId",
    "Conversation.StartTime",
    "Conversation.EndTime",
    "Conversation.ConversationDate",
    "Conversation.Merged_content",
    "Conversation.Merged_content_user",
    "Conversation.Merged_content_agent",
    "Conversation.Full_conversation",
    "Conversation.Duration"
)

In [ ]:
table_name = "ckm_conv_processed_raw"  # Replace with your table name
spark.sql(f"DROP TABLE IF EXISTS {table_name}")

In [ ]:
import os
import openai
import json
import time
import ast

# Function to get details from a conversation
def get_details(input_text):
    time.sleep(4)

    openai.api_type = openai_api_type
    openai.api_version = openai_api_version
    openai.api_base = openai_api_base
    openai.api_key =  openai_api_key

    # Construct the prompt for the OpenAI API
    prompt = '''You are a JSON formatter for extracting information out of a single chat conversation. 
            Summarize the conversation, key: summary .
            Is the customer satisfied with the agent interaction (Yes or No), key: satisfied . 
            Identify the sentiment of the customer as (Positive, Negative or Neutral),key : avgSentiment . 
            Identify the origin city of travel,key: OriginCity . 
            Identify the destination city of travel,key : DestinationCity . 
            Identify the single primary complaint of the conversation in 6 words or less,key: Complaint . 
            Identify the single primary compliment of the conversation in 6 words or less,key: Compliment . 
            Identify the name of hotel that was mentioned,key: Hotel . 
            Identify the name of airline if mentioned,key: Airline . 
            Identify the name of the agent,key: AgentName .
            Identify the top 10 key phrases as comma seperated string excluding people names , key: keyPhrases .
            Identify the main topic, key: topic .
            Identify the language of the text using ISO 639 two letter language identifier, key: lang .
            Identify input_text translated to english, return the same text if already in english, key: translated_text .
            Answer in JSON machine-readable format, using the keys from above. 
            Pretty print the JSON and make sure that it is properly closed at the end and do not generate any other content.'''

    # Set maximum number of retries
    max_retries = 5
    attempts = 0

    # Loop until maximum retries are reached
    while attempts < max_retries:
        try:
            # Call OpenAI API with the constructed prompt
            response = openai.ChatCompletion.create(
                engine= "gpt-35-turbo-16k",
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": input_text}
                ]
            )

            # Parse the response from the API
            result = ast.literal_eval(response['choices'][0]['message']['content'])
            # If 'summary' is found in the result, print and return the result
            if 'summary' in result and result['summary'] is not None and result['summary'].strip() != '':
                print(result)
                return result
            else:
                # If 'summary' is not found, increment attempts and try again
                attempts += 1
                print(f"Attempt {attempts} failed. 'summary' not found in result. Trying again.")
                time.sleep(40)
        except Exception as e:
            # If an error occurs, increment attempts and try again
            attempts += 1
            print(f"Attempt {attempts} failed with error: {e}. Trying again.")
            time.sleep(40)

    print("Maximum number of retries reached. Exiting.")
    return {
        'summary': '',
        'satisfied': '',
        'avgSentiment': '',
        'OriginCity': '',
        'DestinationCity': '',
        'Complaint': '',
        'Compliment': "",
        'Hotel': '',
        'Airline': '',
        'AgentName': '',
        'keyPhrases': '',
        'topic': '',
        'lang': '',
        'translated_text': ''
    }

from pyspark.sql import Row

from pyspark.sql.types import *
from pyspark.sql.functions import *

schema = StructType([
             StructField("summary", StringType(), True),
             StructField("satisfied", StringType(), True),
             StructField("avgSentiment", StringType(), True),
             StructField("OriginCity", StringType(), True),
             StructField("DestinationCity", StringType(), True),
             StructField("Complaint", StringType(), True),
             StructField("Compliment", StringType(), True),
             StructField("Hotel", StringType(), True),
             StructField("Airline", StringType(), True),
             StructField("AgentName", StringType(), True),
             StructField("keyPhrases", StringType(), True),
             StructField("topic", StringType(), True),
             StructField("lang", StringType(), True),
             StructField("translated_text", StringType(), True)
         ])

get_detail_udf = udf(lambda content: get_details(content),returnType=schema)

df_processed = selected_df.select(["ConversationId", "ConversationDate", "EndTime","StartTime","Duration","AgentId","AgentName","Team","ResolutionStatus","CallReason",
    "CallerID","Merged_content","Merged_content_agent","Merged_content_user"]) \
                .withColumn("Details", get_detail_udf(col("Merged_content"))) \
                .select(["ConversationId", "ConversationDate", "EndTime","StartTime","Duration","AgentId","AgentName","Team","ResolutionStatus","CallReason","CallerID","Merged_content","Merged_content_agent","Merged_content_user", \
                          col("Details.summary").alias("summary"), \
                          col("Details.satisfied").alias("satisfied"), \
                          col("Details.avgSentiment").alias("avgSentiment"), \
                          col("Details.OriginCity").alias("OriginCity"), \
                          col("Details.DestinationCity").alias("DestinationCity"), \
                          col("Details.Complaint").alias("Complaint"), \
                          col("Details.Compliment").alias("Compliment"), \
                          col("Details.Hotel").alias("Hotel"), \
                          col("Details.Airline").alias("Airline"), \
                          # col("Details.AgentName").alias("AgentName"), \
                          col("Details.keyPhrases").alias("keyPhrases"), \
                          col("Details.topic").alias("topic"), \
                          col("Details.lang").alias("lang"), \
                          col("Details.translated_text").alias("translated_text")]) 
display(df_processed)

In [ ]:
# spark.sql('drop table if exists ckm_conv_processed_raw')
df_processed.write.format('delta').mode('append').option("overwriteSchema", "true").saveAsTable('ckm_conv_processed_raw')

In [ ]:
table_name = "ckm_conv_processed_raw"  # Replace with your table name

# Get the schema of the table
table_schema = spark.table(table_name).schema

# Compare the schema of DataFrame and the table
if str(df_processed.schema) == str(table_schema):
    print("The DataFrame schema matches the table schema.")
else:
    print("The DataFrame schema does not match the table schema.")
    print("\nDataFrame schema:\n" + str(df_processed.schema))
    print("\nTable schema:\n" + str(table_schema))